Before proceeding, ensure you have completed the following:
1. **Follow Step 1 and Step 2** in `README.md`.
2. **Run `datacleaning.sql`** to preprocess your data.
3. **Save the cleaned data** as a CSV file.

Run the following code in Python to check if the data structure matches expectations:

In [19]:
import pandas as pd
from neo4j import GraphDatabase

In [ ]:
# Load CSV
df = pd.read_csv("/your_data_path", header=None, names=column_names) #change the file path)
df.head()

In [ ]:
#check result
df = pd.read_csv("/Users/zhangmanlin/dataset_director.csv", header=None, names=column_names)
df.head()

,userId,imdb_id,title,genres,releaseYear,averageRating,rating,timestamp,tag,relevance,director_names
0,264,tt1232829,21 Jump Street,"Action, Comedy, Crime",2012,7.2,5.0,1543391407,Channing Tatum,0.98800,"Phil Lord, Christopher Miller"
1,647,tt2965466,Last Shift,"Horror, Mystery",2014,5.8,3.0,1551382166,satanism,0.99250,Anthony DiBlasi
2,653,tt1568911,War Horse,"Adventure, Drama, War",2011,7.2,3.5,1538936562,animals - live action,0.99200,Steven Spielberg
3,2165,tt1961175,American Assassin,"Action, Thriller",2017,6.2,1.0,1525740032,propaganda,0.80725,Michael Cuesta
4,2691,tt1232829,21 Jump Street,"Action, Comedy, Crime",2012,7.2,4.5,1442367427,meta,0.98800,"Phil Lord, Christopher Miller"


## Step 2: Prepare Neo4j Environment

1. **Open Neo4j Desktop** and create a **new project**.
   - Launch **Neo4j Desktop** on your machine.
   - Click on the **"New Project"** button.
   - Name your project (e.g., `MovieRecommendation`).

2. **Import your CSV file into the correct import folder** inside your Neo4j database directory.
   - Navigate to the **"Project"** you just created.
   - Open the **"Import"** folder inside the project directory.
   - Place your CSV file (e.g., `dataset_director.csv`) in the **Import folder**.

3. **Start the Neo4j server**:
   - In **Neo4j Desktop**, click on **"Start"** for the database inside your project to start the server.
   - Wait until the status changes to **"Running"**.

Once the Neo4j server is running, you can proceed with data insertion using Neo4j or Python.

## Step 3: Load Data
You can either load data by using Neo4j Browser and Python (Code below)
### **Neo4j Cypher Query for Importing Data**
```cypher
MERGE (u:User {userId: $userId})

MERGE (m:Movie {imdbId: $imdb_id})
ON CREATE SET 
    m.title = $title,
    m.releaseYear = toInteger($releaseYear),
    m.averageRating = toFloat($averageRating)

MERGE (u)-[r:RATED]->(m)
SET r.rating = toFloat($rating), 
    r.timestamp = toInteger($timestamp)

// Handle Genres
WITH m, split($genres, '|') AS genres
UNWIND genres AS genre_name
MERGE (g:Genre {name: trim(genre_name)})
MERGE (m)-[:OF_GENRE]->(g)

// Handle Director(s)
WITH m, split($directors, ',') AS directors
UNWIND directors AS director_name
MERGE (d:Director {name: trim(director_name)})
MERGE (m)-[:DIRECTED_BY]->(d)

// Handle Tags (if available)
WITH m
WHERE $tag IS NOT NULL
MERGE (t:Tag {name: trim($tag)})
MERGE (m)-[:HAS_TAG]->(t)

// Handle Keywords (if relevance >= 0.5)
WITH m
WHERE $tag IS NOT NULL AND $relevance IS NOT NULL AND toFloat($relevance) >= 0.5
MERGE (k:Keyword {name: trim($tag)})
MERGE (m)-[:RELATED_TO {relevance: toFloat($relevance)}]->(k);

### **Neo4j Index Creation for Performance Optimization**
```cypher
CREATE INDEX movie_imdbId IF NOT EXISTS FOR (m:Movie) ON (m.imdbId);
CREATE INDEX director_name IF NOT EXISTS FOR (d:Director) ON (d.name);
CREATE INDEX user_userId IF NOT EXISTS FOR (u:User) ON (u.userId);
CREATE INDEX genre_name IF NOT EXISTS FOR (g:Genre) ON (g.name);
CREATE INDEX tag_name IF NOT EXISTS FOR (t:Tag) ON (t.name);
CREATE INDEX keyword_name IF NOT EXISTS FOR (k:Keyword) ON (k.name);

Or in python, you can run following to achieve same result as above in Neo4j Browser

In [ ]:
# Define Neo4j connection parameters
NEO4J_URI = "neo4j://localhost:7687"  # Update if needed
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = " " #remember to change this here

# Create a connection to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [ ]:
def create_indexes():
    index_queries = [
        "CREATE INDEX movie_imdbId IF NOT EXISTS FOR (m:Movie) ON (m.imdbId);",
        "CREATE INDEX director_name IF NOT EXISTS FOR (d:Director) ON (d.name);",
        "CREATE INDEX user_userId IF NOT EXISTS FOR (u:User) ON (u.userId);",
        "CREATE INDEX genre_name IF NOT EXISTS FOR (g:Genre) ON (g.name);",
        "CREATE INDEX tag_name IF NOT EXISTS FOR (t:Tag) ON (t.name);",
        "CREATE INDEX keyword_name IF NOT EXISTS FOR (k:Keyword) ON (k.name);"
    ]
    
    with driver.session() as session:
        for query in index_queries:
            session.run(query)
        print(" Indexes created successfully!")

# Run index creation
create_indexes()

def insert_data(tx, row):
    query = """
    MERGE (u:User {userId: $userId})

    MERGE (m:Movie {imdbId: $imdb_id})
    ON CREATE SET 
        m.title = $title,
        m.releaseYear = toInteger($releaseYear),
        m.averageRating = toFloat($averageRating)

    MERGE (u)-[r:RATED]->(m)
    SET r.rating = toFloat($rating), 
        r.timestamp = toInteger($timestamp)

    // Handle Genres
    WITH m, split($genres, '|') AS genres
    UNWIND genres AS genre_name
    MERGE (g:Genre {name: trim(genre_name)})
    MERGE (m)-[:OF_GENRE]->(g)

    // Handle Director(s)
    WITH m, split($directors, ',') AS directors
    UNWIND directors AS director_name
    MERGE (d:Director {name: trim(director_name)})
    MERGE (m)-[:DIRECTED_BY]->(d)

    // Handle Tags (if available)
    WITH m
    WHERE $tag IS NOT NULL
    MERGE (t:Tag {name: trim($tag)})
    MERGE (m)-[:HAS_TAG]->(t)

    // Handle Keywords (if relevance >= 0.5)
    WITH m
    WHERE $tag IS NOT NULL AND $relevance IS NOT NULL AND toFloat($relevance) >= 0.5
    MERGE (k:Keyword {name: trim($tag)})
    MERGE (m)-[:RELATED_TO {relevance: toFloat($relevance)}]->(k);
    """

    tx.run(query, userId=row[0], imdb_id=row[1], title=row[2], genres=row[3],
           releaseYear=row[4], averageRating=row[5], rating=row[6], 
           timestamp=row[7], tag=row[8], relevance=row[9], directors=row[10])

#  Insert Data in Batches
def load_data_to_neo4j():
    with driver.session() as session:
        for index, row in df.iterrows():
            session.write_transaction(insert_data, row)
            if index % 100 == 0:
                print(f"Inserted {index} records...")

# Run the data insertion
load_data_to_neo4j()

print("Data successfully loaded into Neo4j!")

Below is three demonstration example queries.

In [8]:
def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, parameters=params)
        return pd.DataFrame([dict(record) for record in result])

In [20]:
recommendation_query = """
MATCH (u:User {userId: $userId})-[r1:RATED]->(m:Movie)
WHERE r1.rating >= 4.0

MATCH (u2:User)-[r2:RATED]->(m)
WHERE u2 <> u AND r2.rating >= 4.0

WITH u, u2, COUNT(DISTINCT m) AS similarity
ORDER BY similarity DESC
LIMIT 5 // Top 5 similar users

MATCH (u2)-[r3:RATED]->(candidate:Movie)
WHERE r3.rating >= 4.0 AND NOT EXISTS((u)-[:RATED]->(candidate))

WITH DISTINCT u, candidate

OPTIONAL MATCH (u)-[:RATED|TAGGED]->(likedMovie:Movie)
WHERE EXISTS((u)-[:RATED {rating: 4.0}]->(likedMovie)) OR EXISTS((u)-[:TAGGED]->(likedMovie))

MATCH (likedMovie)-[:OF_GENRE|DIRECTED_BY|HAS_TAG|RELATED_TO]->(commonEntity)
      <-[:OF_GENRE|DIRECTED_BY|HAS_TAG|RELATED_TO]-(candidate)

WITH candidate, COUNT(DISTINCT commonEntity) AS contentScore, 
     COLLECT(DISTINCT commonEntity.name) AS matchedContent

OPTIONAL MATCH (u)-[:INTERESTED_IN]->(interest:Keyword)<-[:RELATED_TO]-(candidate)
WITH candidate, contentScore, matchedContent,
     COUNT(DISTINCT interest) AS interestScore,
     COLLECT(DISTINCT interest.name) AS matchedInterests

WITH candidate, matchedContent, matchedInterests,
     (contentScore + interestScore) AS finalRecommendationScore

RETURN candidate.title AS RecommendedMovie,
       matchedContent AS MatchedContent,
       matchedInterests AS MatchedInterests,
       finalRecommendationScore AS TotalScore
ORDER BY TotalScore DESC
LIMIT 10;
"""

# Run query with a specific userId
user_id = "6038"
recommendations_df = run_query(recommendation_query, {"userId": user_id})
recommendations_df

,RecommendedMovie,MatchedContent,MatchedInterests,TotalScore
0,Slumdog Millionaire,"[social commentary, violence, emotional, roman...","[music, romance, predictable, cinematography, ...",139
1,Up,"[funny, rainy day watchlist, comedy, emotional...","[predictable, funny, rainy day watchlist, come...",134
2,Arrival,"[slow, boring, cinematography, thought-provoki...","[predictable, cinematography, obvious, overrat...",134
3,Mad Max: Fury Road,"[surreal, special effects, great soundtrack, c...","[feminism, 2015, action, cinematography, post ...",132
4,The Social Network,"[funny, witty, soundtrack, friendship, adapted...","[music, cinematography, funny, based on a true...",126
5,Prometheus,"[Watched, Michael Fassbender, philosophical, h...","[religion, bad plot, predictable, dialogue, sc...",119
6,Moonrise Kingdom,"[funny, slow, surreal, fantasy, small town, ro...","[fantasy, romance, cinematography, funny, dial...",119
7,Pacific Rim,"[romance, silly, cinematography, ending, sci-f...","[action, romance, bad plot, cinematography, ov...",117
8,500 Days of Summer,"[Funny, slow, romance, great soundtrack, quirk...","[music, romance, overrated, humorous, Funny, q...",115
9,The Imitation Game,"[romance, boring, blu-ray, England, history, L...","[history, 2015, romance, London, boring, based...",113


In [21]:
underrated_query = """
MATCH (u:User {userId: $userId})-[r:RATED]->(m:Movie)

// Convert 10-scale rating explicitly to 5-scale
WITH m, r, (m.averageRating / 2.0) AS convertedAvgRating

// Explicitly round converted rating to nearest 0.5 increment
WITH m, r, 
     round(convertedAvgRating * 2.0) / 2.0 AS roundedAvgRating

// Find movies rated significantly higher by user (≥1.0 point above rounded avg)
WHERE (r.rating - roundedAvgRating) >= 1.0

RETURN m.title AS Movie,
       roundedAvgRating AS RoundedAverageRating,
       r.rating AS UserRating,
       (r.rating - roundedAvgRating) AS RatingDifference
ORDER BY RatingDifference DESC;
"""

# Run query with a specific userId
user_id = "264"
underrated_df = run_query(underrated_query, {"userId": user_id})

underrated_df

,Movie,RoundedAverageRating,UserRating,RatingDifference
0,They Came Together,3.0,5.0,2.0
1,Get Hard,3.0,5.0,2.0
2,21 Jump Street,3.5,5.0,1.5
3,That's My Boy,3.0,4.5,1.5
4,Unfinished Business,2.5,4.0,1.5
5,The Campaign,3.0,4.5,1.5
6,Bad Teacher,3.0,4.0,1.0
7,The Interview,3.5,4.5,1.0


In [22]:
oscar_query = """
// Step 1: Calculate content-based match score
MATCH (u:User {userId: $userId})-[r:RATED]->(m:Movie)
WHERE r.rating >= 4.0

MATCH (m)-[:OF_GENRE|DIRECTED_BY|HAS_TAG|RELATED_TO]->(commonEntity)
      <-[:OF_GENRE|DIRECTED_BY|HAS_TAG|RELATED_TO]-(recMovie:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(recMovie))

WITH recMovie, COUNT(DISTINCT commonEntity) AS matchScore, 
     COLLECT(DISTINCT commonEntity.name) AS matchedEntities

// Step 2: Calculate interest-based match score
OPTIONAL MATCH (u)-[:INTERESTED_IN]->(interest:Keyword)<-[:RELATED_TO]-(recMovie)
WITH recMovie, matchScore, matchedEntities,
     COUNT(DISTINCT interest) AS interestScore,
     COLLECT(DISTINCT interest.name) AS matchedKeywords

// Step 3: Check for Oscar-related tags
OPTIONAL MATCH (recMovie)-[:HAS_TAG]->(oscarTag:Tag)
WHERE oscarTag.name CONTAINS 'Oscar' OR oscarTag.name CONTAINS 'Nominee' OR oscarTag.name CONTAINS 'Best Picture'

// Step 4: Assign Oscar bonus points
WITH recMovie, matchScore, matchedEntities, interestScore, matchedKeywords,
     CASE WHEN oscarTag IS NOT NULL THEN 10 ELSE 0 END AS oscarBonus

// Step 5: Calculate final recommendation score
WITH recMovie, matchedEntities, matchedKeywords,
     (matchScore + interestScore + oscarBonus) AS totalScore

RETURN recMovie.title AS RecommendedMovie,
       matchedEntities AS MatchedEntities,
       matchedKeywords AS MatchedKeywords,
       totalScore AS RecommendationScore
ORDER BY RecommendationScore DESC
LIMIT 10;
"""

# Run the corrected query
user_id = "6038"  # Replace with the target user's ID
oscar_df = run_query(oscar_query, {"userId": user_id})

oscar_df

,RecommendedMovie,MatchedEntities,MatchedKeywords,RecommendationScore
0,Arrival,"[touching, slow, boring, plot twist, plot hole...","[predictable, cinematography, obvious, overrat...",131
1,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
2,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
3,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
4,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
5,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
6,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
7,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
8,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
9,Slumdog Millionaire,"[romance, heartwarming, great soundtrack, unre...","[music, romance, predictable, cinematography, ...",121
